<a href="https://colab.research.google.com/github/tobiasgobel/VQE_project/blob/master/VQE_Clifford_grid_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from sympy.series import series
from scipy.optimize import minimize, NonlinearConstraint
import time
import itertools
from sympy import symbols, Matrix, SparseMatrix, cos, sin, expand, lambdify, O
from sympy.utilities.iterables import multiset_permutations

from functools import *
from operator import *
import scipy
import sympy
import numpy as np
import random
import math
from numba import jit


In [3]:
from functools import wraps
from time import time

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r  took: %2.4f sec' %(f.__name__, te-ts))
        return result
    return wrap

Functions

We denote the ansatz as following:
ansatz = ["X0Y1","X1Y2","X0X1Y2"]

In [4]:

def pauli_on_pauli(p1,p2):
    
    if p1 == 'X' and p2 == 'Y':
        return 1j, 'Z'
    elif p1 == 'X' and p2 == 'X':
        return 1, 'I'
    elif p1 == 'Y' and p2 == 'Y':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'Z':
        return 1, 'I'
    elif p1 == 'Z' and p2 == 'X':
        return 1j, 'Y'
    elif p1 == 'Z' and p2 == 'Y':
        return -1j, 'X'
    elif p1 == 'I':
        return 1, p2
    elif p2 == 'I':
        return 1, p1
    else:
        a, p = pauli_on_pauli(p2,p1)
        return -1*a, p

def single_pauli_action(pauli, spin):
    
    if pauli=='X':
        return((spin+1)%2, 1)
    elif pauli=='Y':
        return((spin+1)%2, 1j*(-1)**spin)
    elif pauli=='Z':
        return(spin, (-1)**spin)
    elif pauli=='I':
        return(spin, 1)
    else:
        print('wrong pauli!')
        return(None)

def findCombinationsUtil(li, arr, index, num, reducedNum):
    z = []
    if (reducedNum < 0): 
        return; 
    if (reducedNum == 0): 
  
        for i in range(index): 
            z = z + [arr[i]]
        li.append(z) 
        return;

    prev = 1 if (index == 0) else arr[index - 1]; 
  
    for k in range(prev, num + 1): 
          

        arr[index] = k; 
  
        findCombinationsUtil(li,arr, index + 1, num,  
                                 reducedNum - k); 
    return li

def k_all(N, generators, order): 
      
    # array to store the combinations 
    # It can contain max n elements
    out = []
    k_length = len(generators)
    for k in range(1, order+1):
        arr = [0] * k;
        output = []
        a =  findCombinationsUtil([], arr, 0, k, k);
        for i in a:
            if len(i)<= k_length:
                i = i.extend((k_length-len(i))*[0])
        for j in a:
            if len(j) == k_length:
#                 if k_vector(N, interactions,j).state()[1] != N*[0]:
                output = output + list(multiset_permutations(j))
        out =  out+ output
    return [tuple(p) for p in [[0]*k_length] + out]

def power_product(x,y):
    out = 1
    for i in range(len(x)):
         out*= x[i]**y[i]
    return out



In [5]:
class pauli:
  def __init__(self,string, N, factor = 1):
    self.string = string
    self.factor = factor
    self.N = N
    self.starting_state = np.array([0]*self.N)

  def __str__(self):
    return self.string+".   factor: "+str(self.factor)
    
  #define multiplying by a constant (on left hand side)
  def __rmul__(self, c):
    self.factor *= c
    return self

  #define the power of a pauli string
  def __pow__(self, c): 
    C = pauli("I0",self.N)
    for i in range(abs(c)):
      C = C*self
    return C

  #define multiplying two pauli strings
  def __mul__(self, x):
    pos1, pauli1 = self.split()
    pos2, pauli2 = x.split()
    factor = self.factor*x.factor
    string = ""
    counter1 =0
    counter2 =0

    for j in range(self.N):
      end1 = counter1 == len(pos1)
      end2 = counter2 == len(pos2)

      if not end1 and not end2:
        if int(pos1[counter1]) == j and int(pos2[counter2]) == j:
          a, p= pauli_on_pauli(pauli1[counter1],pauli2[counter2])
          factor *= a
          string+= p+str(j)
          counter1+=1
          counter2+=1
        elif int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
        elif int(pos2[counter2]) == j:
          string+=pauli2[counter2]+str(j)
          counter2+=1
      elif not end1:
        if int(pos1[counter1]) == j:
          string+=pauli1[counter1]+str(j)
          counter1+=1
      elif not end2:
          if int(pos2[counter2]) == j:
            string+=pauli2[counter2]+str(j)
            counter2+=1
      else:
        pass
      
    return pauli(string, self.N, factor)

  #calculate resulting state of paulistring when acted upon initial_state  
  def state(self, initial_state = 0):
    pos, pauli = self.split()
    init_state = self.starting_state + initial_state
    a = self.factor
    for j in range(len(pos)):
      Pauli = pauli[j]
      spin = init_state[int(pos[j])]
      new_spin, factor = single_pauli_action(Pauli,spin)
      init_state[int(pos[j])] = new_spin
      a *= factor
    return a, tuple(init_state)

    
#creating lists of operators and corresponding positions
  def split(self):
    pauli_lst = []
    pos_lst = []
    prev_int = False
    for k in self.string:
        if k.isdigit():
            if not prev_int:
                pos_lst.append(k)
            else:
                pos_lst[-1] += k
            prev_int = True
        else:
            pauli_lst.append(k)
            prev_int = False
    return pos_lst, pauli_lst
  

  

In [14]:
#gives result of transformation exp(-i*T1)*T2*exp(i*T2)
def Clifford_map(T1, T2, reversed_arguments = True):
  T1T2 = T1*T2
  T2T1 = T2*T1

  if T1T2.factor == T2T1.factor:
    if reversed_arguments:
      return T1
    else:
      return T2
  elif T1T2.factor == -T2T1.factor:
    if reversed_arguments:
      return -1j*T2T1
    else:
      return -1j*T1T2
  else:
    return "something wrong here"


#returns list of pauli objects that are the result 
#of pulling all clifford gates to the left
def pull_cliffords_through(ansatz, K, N):
  T_K = [ansatz[0]]
  
  for j in range(1, len(ansatz)):
    T = ansatz[j]
    for i in range(j-1,-1,-1):
      for _ in range(K[i]):
        T = Clifford_map(T,ansatz[i])
    T_K += [T] 
  return T_K





In [120]:
@timing
def s_dict(N, ansatz, K, order):
  start = time()
  s_dict = {} #keys: possible bitstrings, values dictionary with orders
  T_K = pull_cliffords_through(ansatz, K, N)
  K_all = k_all(N, ansatz, order)

  for i in K_all: #loop through all 
    
    #calculate state that is produced by T_i
    pauli_string = power_product(T_K[::-1], i[::-1])
    factor, state = pauli_string.state()

    #calculate magnitude of term
    factorial = np.prod(np.array([math.factorial(j) for j in i]))
    term = (1j)**sum(i)*factor/factorial
    #check whether binary string is in dictionary, otherwise add
    if state not in s_dict:
      s_dict[state] = {}
    if sum(i) not in s_dict[state]:
      s_dict[state][sum(i)] = ([list(i)],[term])
    else:
      current = s_dict[state][sum(i)]
      current[0].append(list(i))
      current[1].append(term)
  for st in s_dict:
    for som in s_dict[st]:
      lst = s_dict[st][som]
      s_dict[st][som] = (np.array(lst[0]),np.array(lst[1]))
  return s_dict


def G_k(N, H, ansatz, K):
  g_k = []

  #Initialize list of Clifford gates with respective power of K.
  G_K = []
  for i in range(len(K)):
    G_K += [np.sign(K[i])*ansatz[i]]*abs(K[i])
  for P in H:
    # G_K = [ansatz[i]**K[i] for i in range(len(K))]
    #Apply nested Clifford Map to obtain G^-K P_a G^K
    paulistring = reduce(Clifford_map, [P]+G_K[::-1])
    g_k += [paulistring]
  return g_k

@jit(nopython=True)
def dict_add(k,values, thetas):
  sum = 0
  for i in range(k.shape[0]):
    product = 1
    for j in range(k.shape[1]):
      product*=thetas[j]**k[i,j]

    sum += product*values[i]
  return sum

In [121]:

def energy(thetas,ansatz, s_dict,G_K, order):
  E = 0
  s_dict1 = s_dict
  for paulistring in G_K: #loop through terms in Hamiltonian
    E_a = 0

    #loop over basis states
    for s in s_dict1:
      E_a_s = 0
    
      #Calculate G^-K P_a G^K |s>
      a, state = paulistring.state(s)

      #Define contributions of |s> and |s'>
      psi_s1 = s_dict1[s]
      psi_s2 = s_dict1[state]
      #Double for loop to take the right orders in perturbation theory
      for o1 in psi_s1:
        for o2 in range(order - o1 +1):
          if o2 in psi_s2:
            A = dict_add(psi_s1[o1][0],psi_s1[o1][1],thetas)
            B = dict_add(psi_s2[o2][0],psi_s2[o2][1],thetas)
            E_a_s += A*np.conj(B)

      E_a_s *= a
      E_a += E_a_s
    
    E += E_a
  return np.real(E)


In [138]:
class K_hopping:
  def __init__(self,N, ansatz, H, order, K_init):
    self.N = N
    self.ansatz = ansatz
    self.H = H
    self.order = order
    self.K = K_init
    self.path = [K_init.copy()]
    self.len = len(ansatz)
    self.E_path = []

  def evaluate(self):
    #init thetas
    initial_guess = [.001]*self.len

    #initializing s-dictionary and clifford gates
    s = s_dict(self.N, self.ansatz, self.K, self.order)
    G_K = G_k(self.N, self.H, self.ansatz, self.K)
    print("evl",self.K)
    
    #scipy minimize around self.K
    result = scipy.optimize.minimize(energy, initial_guess,jac = False, args = (self.ansatz,s,G_K,self.order))
    print(result)
    return result

  def step(self, result):
    #determine index largest angle
    thetas = result.x
    index = np.argmax(np.abs(thetas))
    print(result.x,result.fun)
    if thetas[index] >= np.pi/8:
      self.K[index] += 1
    elif thetas[index] <= -np.pi/8:
      self.K[index] -= 1
    else:
      return False
    self.path += [self.K.copy()]
    print(self.K)
    return True

  @timing
  def hopping(self,iters=10):
    next = True
    for _ in range(iters):
      result = self.evaluate()
      next = self.step(result)
      if not next:
        return result
    return result





In [139]:
N = 4
ansatz = [pauli("X0Y1",N),pauli("X1Y2",N),pauli("X2Y3",N)]

Z_h = -.1
X_h = -.2
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h),pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h)]

K = [0]*3
order = 10

instance = K_hopping(N,ansatz, H, order, K)
print(instance.hopping(), instance.path)

func:'s_dict'  took: 0.0541 sec
evl [0, 0, 0]
      fun: -0.6732438530089973
 hess_inv: array([[1.08462793, 0.33971072, 0.08462894],
       [0.33971072, 1.26696743, 0.33970775],
       [0.08462894, 0.33970775, 1.08462994]])
      jac: array([1.13248825e-06, 4.32133675e-07, 1.12503767e-06])
  message: 'Optimization terminated successfully.'
     nfev: 24
      nit: 5
     njev: 6
   status: 0
  success: True
        x: array([-0.46088969, -0.51356427, -0.46088969])
[-0.46088969 -0.51356427 -0.46088969] -0.6732438530089973
[0, -1, 0]
func:'s_dict'  took: 0.0464 sec
evl [0, -1, 0]
      fun: -0.4
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
      jac: array([0., 0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([0.001, 0.001, 0.001])
[0.001 0.001 0.001] -0.4
func:'hopping'  took: 0.2062 sec
      fun: -0.4
 hess_inv: array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])
   

In [128]:
N = 10
ansatz = [pauli("X0Y1",N),pauli("X1Y2",N),pauli("X2Y3",N),pauli("X3Y4",N),pauli("X4Y5",N),pauli("X5Y6",N),pauli("X6Y7",N),pauli("X7Y8",N),pauli("X8Y9",N)]
Z_h = -1
X_h = -1
H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h),pauli("Z3",N,Z_h),pauli("Z4",N,Z_h),pauli("Z5",N,Z_h),pauli("Z6",N,Z_h),pauli("Z7",N,Z_h),pauli("Z8",N,Z_h),pauli("Z9",N,Z_h)]
H+=[pauli("X0X1",N,X_h),pauli("X1X2",N,X_h),pauli("X2X3",N,X_h),pauli("X4X5",N),pauli("X5X6",N),pauli("X6X7",N),pauli("X7X8",N),pauli("X8X9",N)]
K = [0]*9
order = 10
thetas=[0]*9

# N = 3
# ansatz = [pauli("X0Y1",N),pauli("X1Y2",N)]
# H = [pauli("Z0",N,Z_h),pauli("Z1",N,Z_h),pauli("Z2",N,Z_h)]
# H+= [pauli("X0X1",N,X_h),pauli("X1X2",N)]
# order = 4
# thetas = [0,0]
# K = [0,0]

s = s_dict(N, ansatz, K, order)
G_K = G_k(N, H, ansatz, K)
# print(s)

# print([str(i) for i in G_K])

print(energy(thetas, ansatz, s, G_K, order))
# print()

func:'s_dict'  took: 28.5853 sec


/usr/local/lib/python3.7/dist-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'thetas' of function 'dict_add'.

For more information visit https://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "<ipython-input-120-26ada535f089>", line 48:
@jit(nopython=True)
def dict_add(k,values, thetas):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


-10.0


In [118]:
print(127.01835942268372-127.01831936836243)
print(127.01804065704346-127.01831936836243)

4.00543212890625e-05
-0.00027871131896972656


In [129]:
initial_guess = [0]*len(ansatz)
result = scipy.optimize.minimize(energy, initial_guess,jac = False, args = (ansatz,s,G_K,order))

In [130]:
print(result)

      fun: -11.968432487101088
 hess_inv: array([[ 5.61097407e-01,  1.46855190e-02, -4.38869847e-01,
         4.94431381e-05, -4.21177916e-04,  8.28131721e-04,
         1.61780743e-03,  7.54232264e-04, -4.43673169e-04],
       [ 1.46855190e-02,  1.28238091e-01,  1.46784114e-02,
        -6.00456937e-06, -3.37173915e-04,  8.01112829e-04,
         1.37842852e-03,  8.23810585e-04, -3.41721009e-04],
       [-4.38869847e-01,  1.46784114e-02,  5.61162886e-01,
         4.94209307e-05, -4.47374214e-04,  8.24919515e-04,
         1.57600825e-03,  7.51045567e-04, -4.69855149e-04],
       [ 4.94431381e-05, -6.00456937e-06,  4.94209307e-05,
         9.99999965e-01, -3.45227319e-05, -3.50902631e-07,
        -6.45702368e-05, -3.10940136e-07, -3.45003187e-05],
       [-4.21177916e-04, -3.37173915e-04, -4.47374214e-04,
        -3.45227319e-05,  5.60544593e-01,  6.85412339e-03,
         4.18615622e-05,  6.88936357e-03, -4.39419090e-01],
       [ 8.28131721e-04,  8.01112829e-04,  8.24919515e-04,
        -

In [85]:
current =([[2, 1]], [(0.5-0j)])
current[0].append([[1,2]])
print(current)

([[2, 1], [[1, 2]]], [(0.5+0j)])


In [109]:
start = time()
print(f'Code took {time()-start}')

Code took 6.246566772460938e-05
